In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# again, fashionMNIST dataset
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 3 layer from before
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [12]:
# hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [13]:
# loss function
loss_fn = nn.CrossEntropyLoss()

In [14]:
# optimizer (SGD, Adam, etc.)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Testing, no gradient updates
    with torch.no_grad():
        for X, y in dataloader:
            # only forward pass
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307181  [   64/60000]
loss: 2.293786  [ 6464/60000]
loss: 2.275558  [12864/60000]
loss: 2.266254  [19264/60000]
loss: 2.237830  [25664/60000]
loss: 2.213578  [32064/60000]
loss: 2.226292  [38464/60000]
loss: 2.184302  [44864/60000]
loss: 2.179261  [51264/60000]
loss: 2.151503  [57664/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 2.142893 

Epoch 2
-------------------------------
loss: 2.157106  [   64/60000]
loss: 2.144326  [ 6464/60000]
loss: 2.087234  [12864/60000]
loss: 2.101060  [19264/60000]
loss: 2.041438  [25664/60000]
loss: 1.988272  [32064/60000]
loss: 2.022054  [38464/60000]
loss: 1.935778  [44864/60000]
loss: 1.934989  [51264/60000]
loss: 1.867010  [57664/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 1.862461 

Epoch 3
-------------------------------
loss: 1.901399  [   64/60000]
loss: 1.867357  [ 6464/60000]
loss: 1.750870  [12864/60000]
loss: 1.787445  [19264/60000]
loss: 1.675647  [25664/60000]
loss: 1.639302  [32064/600